# 벡터의 유사도 (Vector Similarity)
문장이나 문서의 유사도를 구하는 작업!!!
- 사람들이 인식하는 문서의 유사도
    - 문서들 간에 동일한 단어, 비슷한 단어가 얼마나 공통적으로 많이 사용되었는지 의존
- 문서의 유사도의 성능은 각 문서의 단어를 어떤 방법을 수치화하여 표현했는지, 문서 간의 단어들의 차이를 어떤 방법(유클리드 거리, 코사인 유사도 등)으로 계산했는지에 달려있다.

## 1. 코사인 유사도
- 두 벡터 간의 코사인 각도를 이용하여 구할 수 있는 두 벡터의 유사도
- -1~1 사이의 값을 가진다
- 예시
    - 저는 사과 좋아요 
    - 저는 바나나 좋아요
    - 저는 바나나 좋아요 저는 바나나 좋아요

    
![table]("./1.png")

In [2]:
# Numpy 이용하여 코사인 유사도 구하기
import numpy as np
from numpy import dot
from numpy.linalg import norm

def cos_sim(A, B):
    return dot(A,B) / (norm(A)*norm(B))

doc1= np.array([0,1,1,1])
doc2= np.array([1,0,1,1])
doc3= np.array([2,0,2,2])

print("문서 1, 2 의 유사도 :", cos_sim(doc1,doc2))
print("문서 2, 3 의 유사도 :", cos_sim(doc2,doc3))
print("문서 1, 3 의 유사도 :", cos_sim(doc1,doc3))

문서 1, 2 의 유사도 : 0.6666666666666667
문서 2, 3 의 유사도 : 1.0000000000000002
문서 1, 3 의 유사도 : 0.6666666666666667


### 유사도를 이용한 추천 시스템 구현하기
- 좋아하는 영화를 입력하면, 해당 영화의 줄거리와 유사한 줄거리의 영화를 추천해주는 시스템


In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

data = pd.read_csv('movies_metadata.csv', low_memory=False)
data.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [9]:
# Data 상위 2만개 추출
data = data.head(20000)

# TF-IDF 는 데이터에 Null이 들어가있는 경우, 에러가 발생한다
# Null 이 있는지 확인하기
print("Overview 열의 결측값(NaN)의 수 ", data['overview'].isnull().sum())

# 데이터 채우기
data['overview'] = data['overview'].fillna('')

# TF-IDF 행렬 만들기
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(data['overview'])

# 20,000 개의 영화를 표현하기 위해 47,487 개의 단어들이 사용되었음
print("TF-IDF 행렬 크기(shape) :", tfidf_matrix.shape)

# 코사인 유사도 구하기
cosine_sim  = cosine_similarity(tfidf_matrix, tfidf_matrix)
# 각 영화간의 상호 연관성을 가진 코사인 유사도 행렬을 만들어줌
print("코사인 유사도 연산 결과 :", cosine_sim.shape)


Overview 열의 결측값(NaN)의 수  0
TF-IDF 행렬 크기(shape) : (20000, 47487)
코사인 유사도 연산 결과 : (20000, 20000)


In [10]:
# 영화 타이틀 = key, 영화 인덱스 = value
title_to_index = dict(zip(data['title'], data.index))

# 영화 제목의 인덱스를 리턴
idx = title_to_index['Father of the Bride Part II']
print(idx)

4


In [11]:
# 영화를 선택하면 코사인 유사도로 overview 가 유사한 10개 영화를 찾아내주는 함수
def get_recommendation(title, cosine_sim=cosine_sim):
    # 선택한 영화의 인덱스 가져오기
    idx = title_to_index[title]
    # 해당 영화와의 모든 코사인 유사도 가져오기
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # 유사도에 따라 영화를 정렬
    sim_scores = sorted(sim_scores, key=lambda x:x[1], reverse=True)
    
    # 가장 유사한 10개 영화 가져오기 ( 이떄 가장 큰건 본인이기 때문에 1번째부터 가져오기)
    sim_scores = sim_scores[1:11]
    
    # 가장 유사한 10개 영화의 인덱스 가져오기
    movie_index = [idx[0] for idx in sim_scores]
    
    # 영화 제목을 반환
    return data['title'].iloc[movie_index]

In [12]:
get_recommendation('The Dark Knight Rises')

12481                            The Dark Knight
150                               Batman Forever
1328                              Batman Returns
15511                 Batman: Under the Red Hood
585                                       Batman
9230          Batman Beyond: Return of the Joker
18035                           Batman: Year One
19792    Batman: The Dark Knight Returns, Part 1
3095                Batman: Mask of the Phantasm
10122                              Batman Begins
Name: title, dtype: object